In [87]:
import os
import pandas as pd
import numpy as np
import umap
import hdbscan
import matplotlib.pyplot as plt
import data
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from  sklearn.metrics.pairwise import cosine_similarity
from random import sample

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [71]:
data_path = "data/negotiate"
lines = []
file_name = os.path.join(data_path, 'data.txt')
with open(file_name, 'r') as f:
    for line in f:
        lines.append(line.strip())
g =  [line[:12] for line in lines]
o =  [line[-12:] for line in lines]
selection = [line[12:-12].split('<eos>')[-2].split('<selection>')[1] for line in lines]
rew = [line[12:-12].split('<eos>')[-1][8:] for line in lines]
dialog = [line[12:-12].split('<eos>')[:-2] for line in lines]

In [72]:
df = pd.DataFrame({'g':g, 'dialog':dialog, 'selection':selection, 'reward':rew, 'outcome':o})
df.head()
df.to_csv("nego_games.csv", index=False)

In [73]:
deals = [s.strip() != 'no agreement' for s in selection]

In [74]:
sum(deals)/2

5460.0

In [75]:
df['books']  = [g.split()[0] for g in df['g']]
df['hats']  = [g.split()[2] for g in df['g']]
df['balls']  = [g.split()[4] for g in df['g']]



In [76]:
df


,g,dialog,selection,reward,outcome,books,hats,balls
0,1 0 4 2 1 2,[YOU: i would like 4 hats and you can have the...,item0=0 item1=4 item2=0,8 agree,1 4 4 1 1 2,1,4,1
1,1 4 4 1 1 2,[THEM: i would like 4 hats and you can have th...,item0=1 item1=0 item2=1,6 agree,1 0 4 2 1 2,1,4,1
2,1 6 3 0 2 2,[YOU: you can have all the hats if i get the b...,item0=1 item1=3 item2=2,10 disagree,1 2 3 2 2 1,1,3,2
3,1 10 3 0 1 0,[ YOU: hi i would like the book and ball and y...,item0=1 item1=0 item2=0,10 agree,1 2 3 2 1 2,1,3,1
4,1 2 3 2 1 2,[THEM: hi i would like the book and ball and y...,item0=0 item1=3 item2=1,8 agree,1 10 3 0 1 0,1,3,1
...,...,...,...,...,...,...,...,...
12229,1 7 2 0 3 1,"[THEM: one book one hat to me , all else to yo...",item0=0 item1=1 item2=3,3 agree,1 3 2 2 3 1,1,2,3
12230,1 1 1 0 3 3,"[THEM: i need the book and a ball , YOU: how ...",item0=0 item1=0 item2=3,9 agree,1 3 1 1 3 2,1,1,3
12231,1 3 1 1 3 2,"[YOU: i need the book and a ball , THEM: how ...",item0=1 item1=1 item2=0,4 agree,1 1 1 0 3 3,1,1,3
12232,1 0 2 1 2 4,"[THEM: i want the book and both hats , you get...",item0=0 item1=0 item2=2,8 agree,1 4 2 2 2 1,1,2,2


In [77]:
sents = []

for d in df['dialog']:
    for sent in d:
        s = sent.strip()
        if s[:3] == 'YOU':
            sents.append(s[4:].strip())

In [78]:
for s in df['dialog'][1]:
    print(s.split(':')[1].strip())

i would like 4 hats and you can have the rest .
deal


In [79]:
with open('sent.txt', 'w') as f:
    for s in sents:
        f.write(s +'\n')

In [80]:
df = pd.DataFrame({"sentences":sents})

In [81]:
df

,sentences
0,i would like 4 hats and you can have the rest .
1,deal
2,you can have all the hats if i get the book an...
3,hi i would like the book and ball and you can ...
4,ill take the book
...,...
30529,take all the balls rest for me i guess
30530,mmk
30531,sold
30532,"i want the book and both hats , you get the ba..."


In [82]:
import csv


In [83]:
partial = pd.read_csv("dataset100.csv", index_col=0)


In [84]:
partial

,sentences,greeting,need:books,need:hats,need:balls,offer:books,offer:hats,offer:balls,inquiry,agreement
index,,,,,,,,,,
0,i would like 4 hats and you can have the rest .,False,NaN,4,NaN,all,0,all,NaN,NaN
1,deal,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full
2,you can have all the hats if i get the book an...,False,1,NaN,all,NaN,all,NaN,NaN,NaN
3,hi i would like the book and ball and you can ...,True,1,NaN,1,NaN,all,NaN,NaN,NaN
4,ill take the book,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
30529,take all the balls rest for me i guess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30530,mmk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30531,sold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
